 # Alle Karteringen omzetten

 Dit notebook voert alle karteringen via veg2hab naar een habitattypekartering. Instructies:

 - Kopieer de provinciefolders uit de OneDrive: `Spheer AI/Spheer AI - General/Projecten/Veg_2_Hab/Data/Vegetatiekarteringen` naar `testing/vegetatiekarteringen` (of naar iets anders, dan moet je `originele_karteringen_prefix` aanpassen onder **config**)
 - Eerst moeten sommige shapefiles opgeschoond worden. Dit kan met het `handmatig_opschonen_shapefiles.ipynb` notebook. Run deze als dat niet al eens gedaan is.
 - Zorg dat de paden `originele_karteringen_prefix` en `opgeschoonde_shapefiles_prefix` kloppen en overeenkomen met dezelfde paden in het `handmatig_opschonen_shapefiles.ipynb`. Deze prefixes zullen voor de paden vermeld in de spreadsheets geplakt worden.
 - Run de cellen. Deze zullen:
   - Imports importen en het bestaan van de paden controleren.
   - De voor het omzetten benodigde klassen instantieren; de was-wordt lijst, de definitietabel en de FGR kaart.
   - Spreadsheets met daarin configuraties voor iedere kartering inlezen.
   - Alle karteringen inladen in een grote dict.
   - De volgende stappen een voor een op iedere kartering uitvoeren:
     - De was-wordt lijst toepassen (toevoegen van VvN codes aan karteringen met enkel SBB).
     - De definitietabel toepassen (het opstellen van habitatvoorstellen voor ieder vlak).
     - De mitsen checken (eventuele criteria in de habitatvoorstellen checken) en voor ieder vegetatietype een habitatkeuze maken.
     - De kartering formatten als een habitattypekartering en wegschrijven.

 ### Imports etc

In [ ]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_waswordtlijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
from veg2hab.bronnen import FGR, LBK, Bodemkaart
from veg2hab.io.cli import CLIInterface
from veg2hab.enums import WelkeTypologie


CLIInterface.get_instance()

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 8)

originele_karteringen_prefix = '../testing/vegetatiekarteringen/'
opgeschoonde_shapefiles_prefix = '../testing/opgeschoonde_shapefiles/'
output_prefix = '../testing/omgezette_vegetatiekarteringen/'

assert Path(originele_karteringen_prefix).exists(), \
    f"De map {originele_karteringen_prefix} bestaat niet. Zet hier de vegetatiekarteringen uit de onedrive in."

assert Path(opgeschoonde_shapefiles_prefix).exists(), \
    f"De map {opgeschoonde_shapefiles_prefix} bestaat niet. Maak deze eerst aan met het handmatig_opschonen_shapefiles.ipynb notebook."

start_time = pd.Timestamp.now()

 ### Omzetclasses instantieren

In [ ]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_waswordtlijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)

path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_mitsjson = Path("../data/mitsjson.json")
path_in_mozaiekjson = Path("../data/mozaiekjson.json")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_mitsjson, path_in_mozaiekjson, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)

fgr = FGR(Path("../data/bronbestanden/FGR.json"))


 ### Inladen benodigde informatie

 **Overzicht_vegetatiekarteringen.xlsx** is uit de drive en weet van karteringen de provincie, naam, shapefile pad en of het een access of een shapefile kartering is.

 **accesskarteringinfo.xlsx** bevat voor de access karteringen de informatie die nodig is om een access kartering in te laden.

 **shapefilekarteringinfo.xlsx** bevat voor de shapefile karteringen de informatie die nodig is om een shapefile kartering in te laden.


In [ ]:
# info over de karteringen
overzicht = pd.read_excel(
    "../testing/Overzicht_vegetatiekarteringen.xlsx",
    sheet_name="Vegetatiekarteringen",
    engine='openpyxl',
    usecols=[
        "provincie",
        "naam_kartering",
        "path_shapes",
        "landelijke_typologie beschikbaar?",
        "locatie_landelijke_typologie",
    ],
    dtype="string"
)

# benodigde info voor de access karteringen
access_info = pd.read_excel(
    "../testing/accesskarteringinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "path_csvs",
        "elm_id_name",
        "welke_typologie",
    ],
    dtype="string"
)

# benodigde info voor de shapefile karteringen
shape_info = pd.read_excel(
    "../testing/shapefilekarteringinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "ElmID_col",
        "datum_col",
        "opmerking_col",
        "vegtype_col_format",
        "welke_typologie",
        "SBB_col",
        "VvN_col",
        "split_char",
        "perc_col",
    ],
    dtype="string"
)
# Omzetten van pd.NA naar None om te voorkomen dat we overal "if not pd.isna()" moeten doen
# NOTE: dit naar vegkaartering.py?
shape_info = shape_info.applymap(lambda x: None if pd.isna(x) else x)

overzicht = overzicht[overzicht["landelijke_typologie beschikbaar?"] == "Ja"]
# omzetten van ./PROVINCIE/gebied/gebied.shp naar ../testing/opgeschoonde_shapefiles/PROVINCIE/gebied/gebied.shp
overzicht.path_shapes = overzicht.path_shapes.apply(lambda x: opgeschoonde_shapefiles_prefix + x[2:])

access_overzicht = overzicht[overzicht["locatie_landelijke_typologie"] == "access"]
access_overzicht = access_overzicht.join(access_info.set_index("naam_kartering"), on="naam_kartering")
# omzetten van ./PROVINPCIE/gebied/gebied.shp naar ../testing/vegetatiekarteringen/PROVINCIE/gebied/gebied.shp
access_overzicht.path_csvs = access_overzicht.path_csvs.apply(lambda x: originele_karteringen_prefix + x[2:])

shapefile_overzicht = overzicht[overzicht["locatie_landelijke_typologie"] == "in shapes"]
shapefile_overzicht = shapefile_overzicht.join(shape_info.set_index("naam_kartering"), on="naam_kartering")


### Karteringen inladen

Hier worden, adhv configuraties uit de spreadsheets, alle karteringen ingeladen. Als er een probleem is met een kartering, wordt deze overgeslagen en wordt een melding gegeven.

Er zijn vier karteringen die op het moment niet ondersteund worden. 
- NM_Leggelderveld2020
  - Vegetatietype codes zijn rVvN
- NM_Mantingerveld2020
  - Vegetatietype codes zijn rVvN
- NM_Norg_heiden2020
  - Vegetatietype codes zijn rVvN
- NM vegetatiekartering RuitenAa2020
  - Er is geen 1 op 1 relatie voor ElmID in de shapefile en in de database

In [ ]:
karteringen = {}
mislukte_karteringen = []


for row in access_overzicht.itertuples():
    print(f"------ Inlezen {row.naam_kartering}")

    try:
        welke_typologie = WelkeTypologie(row.welke_typologie)
        karteringen[row.naam_kartering] = Kartering.from_access_db(
            Path(row.path_shapes), row.elm_id_name, Path(row.path_csvs), welke_typologie=welke_typologie
        )

    except Exception as e:
        print(e)
        print(f"!!! {row.naam_kartering} wordt overgeslagen !!!")
        mislukte_karteringen.append(row.naam_kartering)
        continue

    print(f"Gelukt!")

for row in shapefile_overzicht.itertuples():
    print(f"------ Inlezen {row.naam_kartering}")

    try:
        welke_typologie = WelkeTypologie(row.welke_typologie)
        karteringen[row.naam_kartering] = Kartering.from_shapefile(
            Path(row.path_shapes),
            vegtype_col_format=row.vegtype_col_format,
            welke_typologie=welke_typologie,
            ElmID_col=row.ElmID_col,
            VvN_col=row.VvN_col.split(row.split_char) if row.VvN_col is not None else [],
            SBB_col=row.SBB_col.split(row.split_char) if row.SBB_col is not None else [],
            rVvN_col=[],
            split_char=row.split_char,
            datum_col=row.datum_col,
            opmerking_col=row.opmerking_col,
            perc_col=row.perc_col.split(row.split_char) if row.perc_col is not None else [],
        )

    except Exception as e:
        print(e)
        print(f"!!! {row.naam_kartering} wordt overgeslagen !!!")
        mislukte_karteringen.append(row.naam_kartering)
        continue

    print(f"Gelukt!")

if len(mislukte_karteringen) > 0:
    print(f"!!! De volgende karteringen zijn niet ingelezen !!!")
    print(mislukte_karteringen)

 ### Karteringen omzetten

#### Was-wordt lijst toepassen

In [ ]:
mislukte_karteringen = []

for key, kartering in karteringen.items():
    print(f"------ Waswordtlijst toepassen {key}")
    try:
        kartering.apply_wwl(wwl)
    except Exception as e:
        print(e)
        print(f"!!! Fout bij {key} !!!")
        mislukte_karteringen.append(key)
        continue
    print(f"Gelukt!")

if len(mislukte_karteringen) > 0:
    print(f"!!! De volgende karteringen hebben een fout bij het toepassen van de was-wordt-lijst !!!")
    print(mislukte_karteringen)


#### Definitietabel toepassen

In [ ]:
mislukte_karteringen = []

for key, kartering in karteringen.items():
    print(f"------ Definitietabel toepassen {key}")
    try:
        kartering.apply_deftabel(dt)
    except Exception as e:
        print(e)
        print(f"!!! Fout bij {key} !!!")
        mislukte_karteringen.append(key)
        continue
    print(f"Gelukt!")

if len(mislukte_karteringen) > 0:
    print(f"!!! De volgende karteringen hebben een fout bij het toepassen van de definitietabel!!!")
    print(mislukte_karteringen)


#### Mitsen/Mozaiekregels checken en Habitatkeuzes bepalen

In [ ]:
mislukte_karteringen = []

for key, kartering in karteringen.items():
    print(f"------ Mitsen/Mozaiekregels checken {key}")
    try:
        mask = kartering.get_geometry_mask()
        lbk = LBK.from_file(Path("../testing/bronbestanden/lbk.shp"), mask=mask)
        bodemkaart = Bodemkaart.from_file(Path("../testing/bronbestanden/bodemkaart.gpkg"), mask=mask)
        kartering.bepaal_mits_habitatkeuzes(fgr, bodemkaart, lbk)
        kartering.bepaal_mozaiek_habitatkeuzes()    
    except Exception as e:
        print(e)
        print(f"!!! Fout bij {key} !!!")
        mislukte_karteringen.append(key)
        continue
    print(f"Gelukt!")

if len(mislukte_karteringen) > 0:
    print(f"!!! De volgende karteringen hebben een fout bij het toepassen checken van de criteria!!!")
    print(mislukte_karteringen)

#### Minimum oppervlak en functionele samenhang checken

In [ ]:
mislukte_karteringen = []

for key, kartering in karteringen.items():
    print(f"------ Minimum oppervlak en functionele samenhang checken {key}")
    try:
        kartering.functionele_samenhang()   
    except Exception as e:
        print(e)
        print(f"!!! Fout bij {key} !!!")
        mislukte_karteringen.append(key)
        continue
    print(f"Gelukt!")

if len(mislukte_karteringen) > 0:
    print(f"!!! De volgende karteringen hebben een fout bij het checken van minimum oppervlak!!!")
    print(mislukte_karteringen)

 ### Karteringen wegschrijven

In [ ]:
for gebied in (karteringen.keys()):
    print(gebied)
    # Vervangen van de shapefile path naar de output path
    p = Path(overzicht[overzicht.naam_kartering == gebied].path_shapes.str.replace(opgeschoonde_shapefiles_prefix, output_prefix).iloc[0])
    p = p.with_suffix('.gpkg')
    p.parent.mkdir(parents=True, exist_ok=True)
    karteringen[gebied].final_format_to_file(p)


### Tijd bezig geweest

In [ ]:
end_time = pd.Timestamp.now()
secs = (end_time - start_time).total_seconds()
print(f"Totale tijd: {secs:.2f} seconden.")

# tel aantal vlakken
n_vlakken = 0
for kartering in karteringen.values():
    n_vlakken += kartering.gdf.shape[0]

# tel aantal keuzes
n_keuzes = 0
for kartering in karteringen.values():
    n_keuzes += kartering.gdf.HabitatKeuze.apply(len).sum()

print(f"Er zijn {n_vlakken} vlakken en {n_keuzes} keuzes omgezet.")

vlakken_per_seconde = n_vlakken / secs
keuzes_per_seconde = n_keuzes / secs

print(f"Dat is {vlakken_per_seconde:.2f} vlakken en {keuzes_per_seconde:.2f} keuzes per seconde.")

seconden_per_kartering = secs / len(karteringen)

print(f"Gemiddeld is dat {seconden_per_kartering:.2f} seconden per kartering.")

### Habitattypestats

In [ ]:
from collections import defaultdict
habdict = defaultdict(int)
for kartering in karteringen.values():
    for row in kartering.gdf.itertuples():
        for keuze in row.HabitatKeuze:
            habdict[keuze.habtype] += 1

habdict = dict(sorted(habdict.items(), key=lambda item: item[1], reverse=True))
habdict